# Potencial Ornitológico Fueguino
### **Autor:** Pablo Jusim

# Script de preprocesamiento de observaciones de iNaturalist

![title](../data/external/logo_inat.png)

Se realizan verificaciones antes de obtener el dataframe final.

La salida de este script será un dataframe de pandas donde cada fila representará una observación de iNaturalist.

El dataframe final tendrá las siguientes **columnas**: id_obs, id_celda, especie, familia, orden, fecha

## Importaciones

In [ ]:
import pandas as pd
from IPython.display import display

## Carga de datos

In [ ]:
archivo = '../data/external/obs_iNat.csv'
try:
    df_base = pd.read_csv(archivo)
    display(df_base.head())
except FileNotFoundError:
    print('No se encuentra el archivo en la ruta ' + archivo)

## Limpieza de datos

### Revisión de datos y eliminación de columnas no útiles

#### Revisión del contenido de columnas con posibles errores y eliminación de las mismas

##### Verificaciones

In [ ]:
# Columna "quality_grade": segun la busqueda realizada en iNat, los datos deberian estar todos en "research"
print(f'quality_grade: {df_base['quality_grade'].unique()}')

# Columna "captive_cultivated": ninguna observacion deberia ser "true" (aves cautivas)
print(f'captive: {df_base['captive_cultivated'].unique()}')

# Columna "private_place_guess": ninguna observacion deberia tener datos ya que la búsqueda solo incluyó ubicaciones públicas
print(f'private_place_guess: {df_base["private_place_guess"].unique()}')

# Columnas "private_latitude" y "private:longitude": ninguna observacion deberia tener datos ya que la búsqueda solo incluyó ubicaciones públicas
print(f'private_latitude: {df_base["private_latitude"].unique()}')
print(f'private_longitude: {df_base["private_longitude"].unique()}')

# Columna "geoprivacy": ninguna observacion deberia tener datos ya que la búsqueda solo incluyó ubicaciones públicas
print(f'geoprivacy: {df_base["geoprivacy"].unique()}')

# Columna "taxon_geoprivacy": las observaciones deberian ser "open" o carecer el dato
print(f'taxon_geoprivacy: {df_base["taxon_geoprivacy"].unique()}')

# Columna "coordinates_obscured": las observaciones deberian ser "false" o carecer el dato
print(f'coordinates_obscured: {df_base["coordinates_obscured"].unique()}')

# Columna "place_town_name": se revisa si hay datos en la columna, sino se elimina
print(f'place_town_name: {df_base["place_town_name"].unique()}')

# Columna "place_county_name": los valores deberian ser los departamentos de TIerras del Fuego
print(f'place_county_name: {df_base["place_county_name"].unique()}')

# Columna "place_state_name": los valores deberian ser "Tierra del Fuego"
print(f'place_state_name: {df_base["place_state_name"].unique()}')

# Columna "iconic_taxon_name": los valores deberian ser "Aves"
print(f'iconic_taxon_name: {df_base["iconic_taxon_name"].unique()}')



In [ ]:
print(f'Filas totales: {len(df_base)}')
print(f'Columnas totales: {len(df_base.columns)}')

### Revisión de valores de filas

##### Precisión

In [ ]:
# Precision de las observaciones: Se descartan observaciones con una precisión menor
# a la mitad del tramaño de la celda (2500 m o más)
df_filtrada_1 = df_base[df_base['positional_accuracy'] < 2500]
print(f'Filas restantes: {len(df_filtrada_1)}')

##### Nombres científicos
Los nombres cientificos constan de dos palabras, el genero y el epíteto específico.
Si hay una tercera palabra, esta puede corresponder a una subespecie.
Tener observaciones de la misma especie con dos o tres palabras puede complicar el conteo

In [ ]:
# Cambiar el nombre de la columna "scientific_name" a "scientific_name_sub"
df_filtrada_2 = df_filtrada_1.rename(columns={'scientific_name': 'scientific_name_sub'})

# Agregar la columna "scientific_name" con el nombre científico de la especie (solo las dos primeras palabras)
df_filtrada_2['scientific_name'] = df_filtrada_2[
    'scientific_name_sub'].apply(
        lambda x: ' '.join(x.split()[:2]) if isinstance(x, str) else x)

df_filtrada_2.head()

##### Eliminación de columnas inútiles o sin paralelo en la otra fuente de datos

In [ ]:
df_filtrada_3 = df_filtrada_2[['common_name', 'scientific_name', 'latitude', 'longitude', 'observed_on']]

# Ver resultado parcial
df_filtrada_3.head()

##### Ver resumen de valores

In [ ]:
# Ver la cantidad de datos faltantes en cada fila
print(df_filtrada_3.isnull().sum())

df_filtrada_3.describe()

No se observan datos faltantes en columnas de importancia para el análisis. No se observan valores de columnas numéricas fuera del rango esperado

## Exportar el data frame resultante

In [ ]:
# Exportar el DataFrame df_filtrada_3
df_filtrada_3.to_csv('../data/raw/data_inat.csv', index=False)